In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from pathlib import Path
import os
from datetime import datetime, date, timedelta
from typing import Any, List, Dict, Union, Tuple
import pandas as pd

## Enable full-width pandas dataframe columns so the paths and filenames always fit
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
# /umms-kdiba/Data/Output/gen_scripts/

import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes
from attrs import define, field, Factory

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList
from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList
from phoglobushelpers.compatibility_objects.Tasks import FatalError, Task, TaskList
from phoglobushelpers.PhoGlobusHelper import TransferRequest

Automatic pdb calling has been turned OFF


In [2]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=Lc86akeKyMM4FpMT74jVH7iIrqSaAhv7pn7wMpSgPtY&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!
My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[df4149a0-9925-11ee-be43-c52a29481bea] Aleinware
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[44727efc-c21b-11ee-89e3-5130436d8581] LNX00052_Ubuntu
[2621bd72-f688-11ee-b6ec-473d136f742f] MM2
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[cee0aaec-3e3b-

# Tasks

In [ ]:
task_id = "a5bba88c-fba0-11ee-9063-1f4fee4027cc"

In [6]:
tasks_list = connect_man.get_tasks()
tasks_list_df = tasks_list.to_dataframe()
tasks_list_df

,bytes_checksummed,bytes_transferred,canceled_by_admin,canceled_by_admin_message,command,completion_time,deadline,delete_destination_extra,destination_endpoint,destination_endpoint_display_name,...,subtasks_retrying,subtasks_skipped_errors,subtasks_succeeded,subtasks_total,symlinks,sync_level,task_id,type,username,verify_checksum
0,0,43206847,None,None,API 0.10 go,2024-08-29T14:38:06+00:00,2024-08-30T14:34:14+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,6949,6949,0,2.0,c322fe6a-6613-11ef-b74f-0f922496edc5,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
1,0,822686210,None,None,API 0.10 go,2024-09-05T13:50:41+00:00,2024-09-06T13:50:29+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,24,24,0,NaN,cf7360d6-6b8d-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
2,0,1144,None,None,API 0.10 go,2024-09-05T13:51:19+00:00,2024-09-06T13:51:17+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,2,2,0,NaN,ec274328-6b8d-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
3,0,1144,None,None,API 0.10 go,2024-09-05T13:51:45+00:00,2024-09-06T13:51:43+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,2,2,0,NaN,fb464b38-6b8d-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
4,0,23859618610,None,None,API 0.10 go,2024-09-05T13:57:25+00:00,2024-09-06T13:53:54+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,64,64,0,NaN,499751ce-6b8e-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
5,0,90092141,None,None,API 0.10 go,2024-09-05T13:54:39+00:00,2024-09-06T13:54:33+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,54,54,0,NaN,610ab1b6-6b8e-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
6,0,164120520,None,None,API 0.10 go,2024-09-05T13:56:30+00:00,2024-09-06T13:55:58+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,1042,1042,0,NaN,933eab10-6b8e-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
7,0,726257,None,None,API 0.10 go,2024-09-05T13:56:10+00:00,2024-09-06T13:56:06+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#cee0aaec-3e3b-11ef-9637-453c3ae125a5,RDLU0039_DibaLabWorkstation,...,0,0,22,22,0,NaN,98743604-6b8e-11ef-aae0-7d6f43498e7d,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
8,0,92129,None,None,API 0.10 go,2024-09-06T22:24:22+00:00,2024-09-07T22:24:05+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,3,3,0,NaN,b9f4e380-6c9e-11ef-b759-0f922496edc5,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
9,0,42817656915,None,None,API 0.10 go,2024-09-11T20:58:13+00:00,2024-09-12T20:20:01+00:00,False,u_upma7q3dzfeqzg454wkb7lyqe4#84991054-07b4-11ed-8d83-a54cf61939f8,Apogee,...,0,0,3836,3836,0,NaN,385db38e-707b-11ef-b4ac-8fef73a45f39,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True


In [7]:
tasks_list_df[tasks_list_df['task_id'] == '82f50134-960c-11ef-aa79-13835fff4299']

,bytes_checksummed,bytes_transferred,canceled_by_admin,canceled_by_admin_message,command,completion_time,deadline,delete_destination_extra,destination_endpoint,destination_endpoint_display_name,...,subtasks_retrying,subtasks_skipped_errors,subtasks_succeeded,subtasks_total,symlinks,sync_level,task_id,type,username,verify_checksum


# Bookmarks

In [4]:
bookmark_list = connect_man.get_bookmarks()
display(bookmark_list.DATA)

lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
lab_DataDen_data_folder_bookmark = Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')

lab_Turbo_collected_outputs_bookmark = Bookmark(bookmark_id='91181e28-f90d-11ee-9222-472b0fe4395a', name='KDIBA Lab Turbo - collected_outputs', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/collected_outputs/')

# Figure Outputs:
lab_Greatlakes_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')
lab_LNX00052_Fedora_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')

## Apogee:
home_Apogee_ProgrammaticDisplayFunctionTesting_bookmark = Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/')
apogee_collected_outputs_folder_bookmark = Bookmark(bookmark_id='965ffa0e-f90d-11ee-9222-472b0fe4395a', name='Apogee - K - collected_outputs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/collected_outputs/')

## `gen_scripts` folders:
apogee_gen_scripts_folder_bookmark = Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/')
lab_Greatlakes_gen_scripts = Bookmark(bookmark_id='99efa634-3ead-11ef-888b-2b3122c1d121', name='Greatlakes gen_scripts', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/gen_scripts/')
# lab_Greatlakes_gen_scripts, apogee_gen_scripts_folder_bookmark

lab_RDLU0039_CollectedOutputs = Bookmark(bookmark_id='2cd4df1e-3f94-11ef-9638-453c3ae125a5', name='RDLU0039_DibaLabWorkstation - Collected Outputs', endpoint_id='cee0aaec-3e3b-11ef-9637-453c3ae125a5', path='/home/halechr/FastData/collected_outputs/')
lab_RDLU0039_data = Bookmark(bookmark_id='20d34336-3f94-11ef-9638-453c3ae125a5', name='RDLU0039_DibaLabWorkstation - FastData', endpoint_id='cee0aaec-3e3b-11ef-9637-453c3ae125a5', path='/home/halechr/FastData/')
lab_RDLU0039_gen_scripts = Bookmark(bookmark_id='295f306c-3f96-11ef-888b-2b3122c1d121', name='RDLU0039_DibaLabWorkstation - gen_scripts', endpoint_id='cee0aaec-3e3b-11ef-9637-453c3ae125a5', path='/home/halechr/FastData/gen_scripts/')

rMBP_collected_outputs = Bookmark(bookmark_id='ec5514de-3f9f-11ef-b849-1dd816fe311b', name='Pho Personal Laptop - collected_outputs', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Users/pho/data/collected_outputs/')
rMBP_gen_scripts = Bookmark(bookmark_id='2d8a8110-43af-11ef-b84e-1dd816fe311b', name='Pho Personal Laptop - gen_scripts', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Users/pho/data/gen_scripts/')
rMBP_data_folder = Bookmark(bookmark_id='e44c77aa-3f9f-11ef-b849-1dd816fe311b', name='Pho Personal Laptop - Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Users/pho/data/')
 
## OUTPUTS: lab_Turbo_data_folder_bookmark, lab_DataDen_data_folder_bookmark, lab_Greatlakes_gen_scripts, apogee_gen_scripts_folder_bookmark, apogee_collected_outputs_folder_bookmark

[Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
 Bookmark(bookmark_id='3453290e-9926-11ee-8c90-fd88ce9321ad', name='Aleinware SmallSSD', endpoint_id='df4149a0-9925-11ee-be43-c52a29481bea', path='/F/'),
 Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
 Bookmark(bookmark_id='4f1229fe-3eba-11ef-9637-453c3ae125a5', name='Apogee - gen_scripts', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/gen_scripts/'),
 Bookmark(bookmark_id='965ffa0e-f90d-11ee-9222-472b0fe4395a', name='Apogee - K - collected_outputs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/K/scratch/collected_outputs/'),
 Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84

# 2024-07-10 - Pulling Greatlakes Batch Run Logs

In [5]:
lab_Greatlakes_gen_scripts = Bookmark(bookmark_id='99efa634-3ead-11ef-888b-2b3122c1d121', name='Greatlakes gen_scripts', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/gen_scripts/')
all_log_file_df, most_recent_only_log_file_df = connect_man.get_greatlakes_gen_scripts_log_files(max_num_day_ago=1)
most_recent_only_log_file_df

earliest_search_day_date: 2024-10-28


,type,parent_path,name,last_modified,size
0,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15,debug_2024-10-29_11-10-14.gl3075.arc-ts.umich.edu.kdiba.gor01.one.2006-6-08_14-26-15.log,2024-10-29 15:29:49+00:00,10668
54,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43,debug_2024-10-29_11-10-11.gl3127.arc-ts.umich.edu.kdiba.gor01.one.2006-6-09_1-22-43.log,2024-10-29 15:28:54+00:00,10660
60,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31,debug_2024-10-29_11-10-11.gl3127.arc-ts.umich.edu.kdiba.gor01.one.2006-6-12_15-55-31.log,2024-10-29 15:28:23+00:00,10668
6,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19,debug_2024-10-29_11-10-14.gl3161.arc-ts.umich.edu.kdiba.gor01.two.2006-6-07_16-40-19.log,2024-10-29 15:29:28+00:00,10668
12,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46,debug_2024-10-29_11-10-14.gl3161.arc-ts.umich.edu.kdiba.gor01.two.2006-6-12_16-53-46.log,2024-10-29 15:29:22+00:00,10668
18,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25,debug_2024-10-29_11-10-14.gl3207.arc-ts.umich.edu.kdiba.pin01.one.11-03_12-3-25.log,2024-10-29 15:29:22+00:00,10628
24,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54,debug_2024-10-29_11-10-14.gl3217.arc-ts.umich.edu.kdiba.pin01.one.fet11-01_12-58-54.log,2024-10-29 15:29:48+00:00,10660
30,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30,debug_2024-10-29_11-10-14.gl3183.arc-ts.umich.edu.kdiba.vvp01.one.2006-4-09_17-29-30.log,2024-10-29 15:29:33+00:00,10668
36,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50,debug_2024-10-29_11-10-14.gl3183.arc-ts.umich.edu.kdiba.vvp01.one.2006-4-10_12-25-50.log,2024-10-29 15:29:33+00:00,10668
42,file,/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54,debug_2024-10-29_11-10-14.gl3185.arc-ts.umich.edu.kdiba.vvp01.two.2006-4-09_16-40-54.log,2024-10-29 15:29:24+00:00,10668


## Copy these most recent logs to the desired endpoint

In [7]:
## Apogee:
gen_scripts_GL_to_Apogee = TransferRequest(source_endpoint=lab_Greatlakes_gen_scripts,
				dest_endpoint=apogee_gen_scripts_folder_bookmark,
				transfer_label='GL->Apogee gen_scripts log Files')

## Lab
gen_scripts_GL_to_Lab = TransferRequest(source_endpoint=lab_Greatlakes_gen_scripts,
				dest_endpoint=lab_RDLU0039_gen_scripts,
				transfer_label='GL->Lab gen_scripts log Files')


## rMBP
gen_scripts_GL_to_rMBP = TransferRequest(source_endpoint=lab_Greatlakes_gen_scripts,
				dest_endpoint=rMBP_gen_scripts,
				transfer_label='GL->rMBP gen_scripts log Files')


## INPUTS: most_recent_only_log_file_df
active_file_df: pd.DataFrame = deepcopy(most_recent_only_log_file_df)
connect_man.perform_copy_files(active_file_df=active_file_df, transfer=gen_scripts_GL_to_Apogee, synchronous_wait=False)
# connect_man.perform_copy_files(active_file_df=active_file_df, transfer=gen_scripts_GL_to_Lab, synchronous_wait=False)
# connect_man.perform_copy_files(active_file_df=active_file_df, transfer=gen_scripts_GL_to_rMBP, synchronous_wait=False)

Submitted transfer task a599af68-43b0-11ef-9641-453c3ae125a5 with 15 files.


{'a599af68-43b0-11ef-9641-453c3ae125a5': GlobusHTTPResponse({
   "DATA_TYPE": "transfer_result",
   "code": "Accepted",
   "message": "The transfer has been accepted and a task has been created and queued for execution",
   "request_id": "aY55l3VE8",
   "resource": "/transfer",
   "submission_id": "a599af69-43b0-11ef-9641-453c3ae125a5",
   "task_id": "a599af68-43b0-11ef-9641-453c3ae125a5",
   "task_link": {
     "DATA_TYPE": "link",
     "href": "task/a599af68-43b0-11ef-9641-453c3ae125a5?format=json",
     "rel": "related",
     "resource": "task",
     "title": "related task"
   }
 })}

# 2024-07-10 - Copying Greatlakes Collected Outputs Exported .CSVs

In [ ]:


top_level_parts_separators = ['-', '__']

def get_only_most_recent_output_files(a_file_df: pd.DataFrame) -> pd.DataFrame:
    """ returns a dataframe containing only the most recent '.err' and '.log' file for each session. 
    
    from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList, get_only_most_recent_log_files
    
    
    """
    df = deepcopy(a_file_df)
    df['last_modified'] = pd.to_datetime(df['last_modified'])

    # Separate .csv and .h5 files
    csv_files = df[df['name'].str.endswith('.csv')]
    h5_files = df[df['name'].str.endswith('.h5')]

    # Get the most recent .err and .log file for each parent_path
    most_recent_csv = csv_files.loc[csv_files.groupby('parent_path')['last_modified'].idxmax()]
    most_recent_h5 = h5_files.loc[h5_files.groupby('parent_path')['last_modified'].idxmax()]

    # Concatenate the results
    most_recent_files = pd.concat([most_recent_csv, most_recent_h5]).sort_values(by=['parent_path', 'last_modified'], ascending=[True, False])
    return most_recent_files



# most_recent_only_csv_file_df = get_only_most_recent_log_files(log_file_df=all_file_df)

In [9]:
# lab_Turbo_collected_outputs_bookmark = Bookmark(bookmark_id='91181e28-f90d-11ee-9222-472b0fe4395a', name='KDIBA Lab Turbo - collected_outputs', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/Output/collected_outputs/')
# all_file_df: pd.DataFrame = connect_man.get_greatlakes_collected_outputs_files(max_num_day_ago=1, filter="name:~*.csv,~*.h5")
all_file_df: pd.DataFrame = connect_man.get_greatlakes_collected_outputs_files(max_num_day_ago=1, filter="name:~*.csv")
all_file_df

earliest_search_day_date: 2024-07-15


,type,parent_path,name,last_modified,size
0,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-16_GL-2006-6-09_1-22-43_merged_complete_epoch_stats_df.csv,2024-07-16 19:29:26+00:00,196773
1,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-16_GL-2006-6-08_21-16-25_merged_complete_epoch_stats_df.csv,2024-07-16 19:15:38+00:00,75320
2,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-16_GL-2006-4-10_12-25-50_merged_complete_epoch_stats_df.csv,2024-07-16 19:02:04+00:00,29098
3,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-16_GL-2006-4-10_12-58-3_merged_complete_epoch_stats_df.csv,2024-07-16 19:01:41+00:00,30703
4,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-16_GL-2006-4-09_17-29-30_merged_complete_epoch_stats_df.csv,2024-07-16 19:01:39+00:00,45294
...,...,...,...,...,...
1845,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-15_GL-2006-6-12_15-55-31_merged_complete_epoch_stats_df.csv,2024-07-16 00:02:45+00:00,44926
1846,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-15_0755PM-kdiba_gor01_two_2006-6-12_16-53-46-(laps_marginals_df)_tbin-0.25.csv,2024-07-15 23:57:26+00:00,8312
1847,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-15_0755PM-kdiba_gor01_two_2006-6-12_16-53-46-(laps_time_bin_marginals_df)_tbin-0.25.csv,2024-07-15 23:57:26+00:00,137459
1848,file,/umms-kdiba/Data/Output/collected_outputs/,2024-07-15_0755PM-kdiba_gor01_two_2006-6-12_16-53-46-(ripple_marginals_df)_tbin-0.025.csv,2024-07-15 23:57:26+00:00,12362


In [10]:
outputs_folder = Path('../EXTERNAL/outputs').resolve()
outputs_folder.mkdir(parents=True, exist_ok=True)
day_date = '2024-07-16'
csv_out_path = outputs_folder.joinpath(f'{day_date}_greatlakes_collected_outputs_csv_files.csv').resolve()
# all_file_df.to_csv('greatlakes_collected_outputs_files.csv')
all_file_df.to_csv(csv_out_path)

## Copy these most recent files to the desired endpoint

In [ ]:
## Apogee:
collected_outputs_GL_to_Apogee = TransferRequest(source_endpoint=lab_Turbo_collected_outputs_bookmark,
				dest_endpoint=apogee_collected_outputs_folder_bookmark,
				transfer_label='GL->Apogee Collected Outputs Files')


## Lab
collected_outputs_GL_to_Lab = TransferRequest(source_endpoint=lab_Turbo_collected_outputs_bookmark,
				dest_endpoint=lab_RDLU0039_CollectedOutputs,
				transfer_label='GL->Lab Collected Outputs Files')


## rMBP
collected_outputs_GL_to_rMBP = TransferRequest(source_endpoint=lab_Turbo_collected_outputs_bookmark,
				dest_endpoint=rMBP_collected_outputs,
				transfer_label='GL->rMBP Collected Outputs Files')


active_file_df: pd.DataFrame = deepcopy(all_file_df)
connect_man.perform_copy_files(active_file_df=active_file_df, transfer=collected_outputs_GL_to_Apogee, synchronous_wait=False)
# connect_man.perform_copy_files(active_file_df=active_file_df, transfer=collected_outputs_GL_to_Lab, synchronous_wait=False)
connect_man.perform_copy_files(active_file_df=active_file_df, transfer=collected_outputs_GL_to_rMBP, synchronous_wait=False)
